In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

***Cargando los datasets***

In [2]:
def load_mesas(file):
    mesas = pd.read_csv(file, sep = '|', low_memory = False)
    mesas['CODIGO_CIRCUITO'] = mesas['CODIGO_CIRCUITO'].astype('str')
    mesas['CODIGO_MESA'] = mesas['CODIGO_MESA'].astype('str')
#     mesas['CONTADOR'] = mesas['CONTADOR'].astype('str')
    return mesas
mesas_p = load_mesas('mesas_totales_paso.dsv')
mesas_p.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CONTADOR,VALOR
0,1,1001,01001000001,0100100001X,100000000000,VB,3
1,1,1001,01001000001,0100100001X,100000000000,VII,0
2,1,1001,01001000001,0100100001X,100000000000,VN,3
3,1,1001,01001000001,0100100001X,100000000000,VR,0
4,1,1001,01001000001,0100100001X,201000000000,VB,0


In [3]:
votos_p = load_mesas('mesas_totales_agrp_politica_paso.dsv')
votos_p.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,VOTOS_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,13,1
1,1,1001,01001000001,0100100001X,100000000000,131,5
2,1,1001,01001000001,0100100001X,100000000000,132,2
3,1,1001,01001000001,0100100001X,100000000000,133,15
4,1,1001,01001000001,0100100001X,100000000000,135,85


In [4]:
agrupaciones_p = pd.read_csv('descripcion_postulaciones_paso.dsv', sep = '|')
agrupaciones_p.head()

,CODIGO_CATEGORIA,NOMBRE_CATEGORIA,CODIGO_AGRUPACION,NOMBRE_AGRUPACION,CODIGO_LISTA,NOMBRE_LISTA
0,801000000000,Diputados Ciudad Autónoma de Buenos Aires,01-504,CONSENSO FEDERAL,000801000000000.01-504.A.000801000000000.01-504,CONSENSO PARA EL FUTURO
1,801000000000,Diputados Ciudad Autónoma de Buenos Aires,13,MOVIMIENTO AL SOCIALISMO,000801000000000.13.A.000801000000000.13,UNIDAD DE LA IZQUIERDA
2,801000000000,Diputados Ciudad Autónoma de Buenos Aires,5,DEMOCRATA CRISTIANO,000801000000000.5.A.000801000000000.5,CIUDAD CELESTE POR LA VIDA Y LA FAMILIA
3,801000000000,Diputados Ciudad Autónoma de Buenos Aires,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD,000801000000000.87.B.000801000000000.87,LISTA DIGNIDAD
4,801000000000,Diputados Ciudad Autónoma de Buenos Aires,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD,000801000000000.87.A.000801000000000.87,OLA CELESTE


***FIltrado del dataset***

In [5]:
mesas_paso = mesas_p.copy()
votos_paso = votos_p.copy()
agrupaciones_paso = agrupaciones_p.copy()

In [6]:
def voto_blanco(df, string, new_column_name):
    df = df[df.CONTADOR == string]
    df = df.rename(columns = {'VALOR': new_column_name})
    df = df.drop(columns = ['CONTADOR'])    
    df = df.reset_index(drop = True)
    return df

In [7]:
mesas_paso = voto_blanco(mesas_paso, 'VB', 'votos_blanco_paso')
mesas_paso

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,votos_blanco_paso
0,1,1001,01001000001,0100100001X,100000000000,3
1,1,1001,01001000001,0100100001X,201000000000,0
2,1,1001,01001000001,0100100001X,301000000000,0
3,1,1001,01001000001,0100100001X,501000000000,0
4,1,1001,01001000001,0100100001X,801000000000,0
...,...,...,...,...,...,...
356100,24,24005,24005000021,2400500483X,224000000000,21
356101,24,24005,24005000021,2400500483X,324000000000,45
356102,24,24005,24005000021,2400500484X,100000000000,2
356103,24,24005,24005000021,2400500484X,224000000000,10


In [8]:
def filter_codigo_categoria(df):
    
    df = df[df.CODIGO_CATEGORIA == 100000000000]
    df = df.reset_index(drop = True)
    return df

In [9]:
mesas_paso = filter_codigo_categoria(mesas_paso)
votos_paso = filter_codigo_categoria(votos_paso)
agrupaciones_paso = filter_codigo_categoria(agrupaciones_paso)

In [10]:
print(len(mesas_paso.CODIGO_MESA.unique()))
print(len(votos_paso.CODIGO_MESA.unique()))

98828
98834


***Cambiando los códigos de las agrupaciones por sus respectivos nombres***

In [11]:
def agrupacion_codigo_nombre(df):
    
    df = df[['CODIGO_AGRUPACION', 'NOMBRE_AGRUPACION']]
    df = pd.DataFrame(df.groupby(['CODIGO_AGRUPACION','NOMBRE_AGRUPACION']).size())
    df = df.reset_index()
    df = df.iloc[:,:2]
    return df

In [12]:
df_agrp_codigos_paso = agrupacion_codigo_nombre(agrupaciones_paso)
df_agrp_codigos_paso

,CODIGO_AGRUPACION,NOMBRE_AGRUPACION
0,13,MOVIMIENTO AL SOCIALISMO
1,131,FRENTE NOS
2,132,FRENTE PATRIOTA
3,133,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
4,135,JUNTOS POR EL CAMBIO
5,136,FRENTE DE TODOS
6,137,CONSENSO FEDERAL
7,36,PARTIDO AUTONOMISTA
8,57,MOVIMIENTO DE ACCION VECINAL
9,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD


In [13]:
votos_paso = pd.merge(votos_paso, df_agrp_codigos_paso, on = 'CODIGO_AGRUPACION', how = 'right')
votos_paso.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,VOTOS_AGRUPACION,NOMBRE_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,13,1,MOVIMIENTO AL SOCIALISMO
1,1,1001,01001000001,0100100002X,100000000000,13,6,MOVIMIENTO AL SOCIALISMO
2,1,1001,01001000001,0100100003X,100000000000,13,4,MOVIMIENTO AL SOCIALISMO
3,1,1001,01001000001,0100100004X,100000000000,13,1,MOVIMIENTO AL SOCIALISMO
4,1,1001,01001000001,0100100005X,100000000000,13,5,MOVIMIENTO AL SOCIALISMO


***Agrupado los datos por agrupación***

In [14]:
def agrupar_datos(df_v, nombre_agrupacion, nombre_columna):
    
    df = df_v[df_v.NOMBRE_AGRUPACION == nombre_agrupacion]
    df = df.rename(columns={'VOTOS_AGRUPACION':nombre_columna})
    df = df.reset_index(drop=True)
    df = df[['CODIGO_MESA', nombre_columna]]
    
    return df

In [15]:
df_consenso_federal = agrupar_datos(votos_paso, 'CONSENSO FEDERAL', 'Votos_consenso_federal_paso')
df_mas = agrupar_datos(votos_paso, 'MOVIMIENTO AL SOCIALISMO', 'Votos_MAS_paso')
df_fit = agrupar_datos(votos_paso, 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'Votos_fit_paso')
df_todos = agrupar_datos(votos_paso, 'FRENTE DE TODOS', 'Votos_Frente_todos_paso')
df_nos = agrupar_datos(votos_paso, 'FRENTE NOS', 'Votos_NOS_paso')
df_frente_patriota = agrupar_datos(votos_paso, 'FRENTE PATRIOTA', 'Votos_frente_patriota_paso')
df_juntos_cambio = agrupar_datos(votos_paso, 'JUNTOS POR EL CAMBIO', 'Votos_juntos_cambio_paso')
df_accion_vecinal = agrupar_datos(votos_paso, 'MOVIMIENTO DE ACCION VECINAL', 'Votos_accion_vecinal_paso')
df_autonomista = agrupar_datos(votos_paso, 'PARTIDO AUTONOMISTA', 'Votos_autonomista_paso')
df_unite = agrupar_datos(votos_paso, 'UNITE POR LA LIBERTAD Y LA DIGNIDAD', 'Votos_unite_paso')

In [16]:
from functools import reduce

dataframes = [df_consenso_federal, df_mas, df_fit, df_todos, df_nos, df_frente_patriota, df_juntos_cambio, df_accion_vecinal,
             df_autonomista, df_unite]

df_paso = reduce(lambda  left,right: pd.merge(left,right,on=['CODIGO_MESA'],
                                            how='outer'), dataframes)
df_paso

,CODIGO_MESA,Votos_consenso_federal_paso,Votos_MAS_paso,Votos_fit_paso,Votos_Frente_todos_paso,Votos_NOS_paso,Votos_frente_patriota_paso,Votos_juntos_cambio_paso,Votos_accion_vecinal_paso,Votos_autonomista_paso,Votos_unite_paso
0,0100100001X,20,1,15,101,5,2,85,1,0,4
1,0100100002X,14,6,9,99,8,1,89,0,0,9
2,0100100003X,14,4,15,106,3,0,94,0,0,6
3,0100100004X,21,1,9,111,3,0,80,0,0,16
4,0100100005X,19,5,10,109,2,0,93,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...
98829,2400500480X,18,0,3,102,31,1,54,0,1,2
98830,2400500481X,24,1,4,118,10,1,29,0,0,3
98831,2400500482X,15,1,5,116,15,1,31,0,1,2
98832,2400500483X,14,0,2,126,14,3,34,0,0,0


***Combinamos el dataset con votos en blanco con el de votos a agrupaciones***

In [17]:
mesas_paso = mesas_paso[['CODIGO_MESA','votos_blanco_paso']]

In [18]:
df_votos_paso = pd.merge(mesas_paso, df_paso, on = 'CODIGO_MESA', how = 'right' )

In [19]:
df_votos_paso.isnull().sum()

CODIGO_MESA                    0
votos_blanco_paso              6
Votos_consenso_federal_paso    0
Votos_MAS_paso                 0
Votos_fit_paso                 0
Votos_Frente_todos_paso        0
Votos_NOS_paso                 0
Votos_frente_patriota_paso     0
Votos_juntos_cambio_paso       0
Votos_accion_vecinal_paso      0
Votos_autonomista_paso         0
Votos_unite_paso               0
dtype: int64

In [20]:
df_votos_paso.dropna(inplace = True)

***Agregamos una columna con el total de votos afirmativos (votos a agrupaciones) y el total de votos validos (votos a agrupaciones + votos en blanco)***

In [21]:
df_votos_paso['Total_votos_afrimativos'] = df_votos_paso.iloc[:,2:].sum(axis=1)
df_votos_paso['Total_votos_validos'] = df_votos_paso.iloc[:,1:12].sum(axis=1)

In [22]:
df_votos_paso.head()

,CODIGO_MESA,votos_blanco_paso,Votos_consenso_federal_paso,Votos_MAS_paso,Votos_fit_paso,Votos_Frente_todos_paso,Votos_NOS_paso,Votos_frente_patriota_paso,Votos_juntos_cambio_paso,Votos_accion_vecinal_paso,Votos_autonomista_paso,Votos_unite_paso,Total_votos_afrimativos,Total_votos_validos
0,0100100001X,3.0,20,1,15,101,5,2,85,1,0,4,234,237.0
1,0100100002X,0.0,14,6,9,99,8,1,89,0,0,9,235,235.0
2,0100100003X,6.0,14,4,15,106,3,0,94,0,0,6,242,248.0
3,0100100004X,9.0,21,1,9,111,3,0,80,0,0,16,241,250.0
4,0100100005X,10.0,19,5,10,109,2,0,93,0,0,10,248,258.0


***Calculamos los porcentajes obtenidos por cada agrupación y en blanco (se calcula dividiendo el total de votos por mesa emitidos a cada agrupación divido el total de votos válidos por mesa***

In [23]:
df_votos_paso.columns

Index(['CODIGO_MESA', 'votos_blanco_paso', 'Votos_consenso_federal_paso',
       'Votos_MAS_paso', 'Votos_fit_paso', 'Votos_Frente_todos_paso',
       'Votos_NOS_paso', 'Votos_frente_patriota_paso',
       'Votos_juntos_cambio_paso', 'Votos_accion_vecinal_paso',
       'Votos_autonomista_paso', 'Votos_unite_paso', 'Total_votos_afrimativos',
       'Total_votos_validos'],
      dtype='object')

In [24]:
porcentaje = lambda x,y: x/y

df_votos_paso['por_consenso_federal_paso'] = porcentaje(df_votos_paso['Votos_consenso_federal_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_fit_paso'] = porcentaje(df_votos_paso['Votos_fit_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_MAS_paso'] = porcentaje(df_votos_paso['Votos_MAS_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_Frente_todos_paso'] = porcentaje(df_votos_paso['Votos_Frente_todos_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_frente_patriota_paso'] = porcentaje(df_votos_paso['Votos_frente_patriota_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_juntos_cambio_paso'] = porcentaje(df_votos_paso['Votos_juntos_cambio_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_accion_vecinal_paso'] = porcentaje(df_votos_paso['Votos_accion_vecinal_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_autonomista_paso'] = porcentaje(df_votos_paso['Votos_autonomista_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_unite_paso'] = porcentaje(df_votos_paso['Votos_unite_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_NOS_paso'] = porcentaje(df_votos_paso['Votos_NOS_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_votos_blanco_paso'] = porcentaje(df_votos_paso['votos_blanco_paso'],df_votos_paso['Total_votos_validos'])

In [25]:
df_votos_paso.head()

,CODIGO_MESA,votos_blanco_paso,Votos_consenso_federal_paso,Votos_MAS_paso,Votos_fit_paso,Votos_Frente_todos_paso,Votos_NOS_paso,Votos_frente_patriota_paso,Votos_juntos_cambio_paso,Votos_accion_vecinal_paso,...,por_fit_paso,por_MAS_paso,por_Frente_todos_paso,por_frente_patriota_paso,por_juntos_cambio_paso,por_accion_vecinal_paso,por_autonomista_paso,por_unite_paso,por_NOS_paso,por_votos_blanco_paso
0,0100100001X,3.0,20,1,15,101,5,2,85,1,...,0.063291,0.004219,0.426160,0.008439,0.358650,0.004219,0.0,0.016878,0.021097,0.012658
1,0100100002X,0.0,14,6,9,99,8,1,89,0,...,0.038298,0.025532,0.421277,0.004255,0.378723,0.000000,0.0,0.038298,0.034043,0.000000
2,0100100003X,6.0,14,4,15,106,3,0,94,0,...,0.060484,0.016129,0.427419,0.000000,0.379032,0.000000,0.0,0.024194,0.012097,0.024194
3,0100100004X,9.0,21,1,9,111,3,0,80,0,...,0.036000,0.004000,0.444000,0.000000,0.320000,0.000000,0.0,0.064000,0.012000,0.036000
4,0100100005X,10.0,19,5,10,109,2,0,93,0,...,0.038760,0.019380,0.422481,0.000000,0.360465,0.000000,0.0,0.038760,0.007752,0.038760


****Ahora que ya tenemos listo el dataset de las elecciones PASO, es necesario hacer el mismo proceso pero para
el dataset de las elecciones generales****

***Cargamos los datos***

In [26]:
def load_df(file):
    
    generales = pd.read_csv(file, sep = '|', low_memory = False)
    generales['CODIGO_CIRCUITO'] = generales['CODIGO_CIRCUITO'].astype('str')
    generales['CODIGO_MESA'] = generales['CODIGO_MESA'].astype('str')
    
    return generales

mesas_g = load_df('mesas_totales.dsv')
votos_g = load_df('mesas_agrp_politicas.dsv')
agrupaciones_g =  pd.read_csv('descripcion_postulaciones.dsv',sep='|')

In [ ]:
mesas_generales = mesas_g.copy()
votos_generales = votos_g,copy()
agrupaciones_generales = agrupaciones_g.copy()

In [27]:
mesas_generales.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CONTADOR,VALOR
0,1,1001,01001000001,0100100001X,100000000000,Voto blanco,5
1,1,1001,01001000001,0100100001X,100000000000,Votos identidad impugnada,0
2,1,1001,01001000001,0100100001X,100000000000,Voto nulo,2
3,1,1001,01001000001,0100100001X,100000000000,Voto Recurrido,0
4,1,1001,01001000001,0100100001X,100000000000,Total de electores de mesas computadas,272


In [28]:
votos_generales.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,CODIGO_LISTA,VOTOS_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,131,000100000000000.131.A.000100000000000.131,3
1,1,1001,01001000001,0100100001X,100000000000,133,000100000000000.133.A.000100000000000.133,14
2,1,1001,01001000001,0100100001X,100000000000,135,000100000000000.135.A.000100000000000.135,115
3,1,1001,01001000001,0100100001X,100000000000,136,000100000000000.136.A.000100000000000.136,114
4,1,1001,01001000001,0100100001X,100000000000,137,000100000000000.137.A.000100000000000.137,14


***Filtramos los datasets***

In [29]:
mesas_generales = mesas_generales[mesas_generales.CONTADOR == 'Voto blanco']
mesas_generales = mesas_generales.reset_index(drop=True)
mesas_generales = mesas_generales.drop(columns = ['CONTADOR'])
mesas_generales = mesas_generales.rename(columns = {'VALOR':'votos_blanco_gen'})

In [30]:
mesas_generales = filter_codigo_categoria(mesas_generales)
votos_generales = filter_codigo_categoria(votos_generales)
agrupaciones_generales = filter_codigo_categoria(agrupaciones_generales)

In [31]:
df_agrp_codigos_gen = agrupacion_codigo_nombre(agrupaciones_generales)
df_agrp_codigos_gen

,CODIGO_AGRUPACION,NOMBRE_AGRUPACION
0,131,FRENTE NOS
1,133,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
2,135,JUNTOS POR EL CAMBIO
3,136,FRENTE DE TODOS
4,137,CONSENSO FEDERAL
5,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD


In [32]:
votos_generales = pd.merge(votos_generales, df_agrp_codigos_gen, on = 'CODIGO_AGRUPACION', how = 'right')
votos_generales.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,CODIGO_LISTA,VOTOS_AGRUPACION,NOMBRE_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,131,000100000000000.131.A.000100000000000.131,3,FRENTE NOS
1,1,1001,01001000001,0100100002X,100000000000,131,000100000000000.131.A.000100000000000.131,2,FRENTE NOS
2,1,1001,01001000001,0100100003X,100000000000,131,000100000000000.131.A.000100000000000.131,2,FRENTE NOS
3,1,1001,01001000001,0100100004X,100000000000,131,000100000000000.131.A.000100000000000.131,5,FRENTE NOS
4,1,1001,01001000001,0100100005X,100000000000,131,000100000000000.131.A.000100000000000.131,1,FRENTE NOS


***Ahora pasamos a agregar a cada agrupación como una columna del dataset***

In [33]:
votos_generales.NOMBRE_AGRUPACION.unique()

array(['FRENTE NOS', 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD',
       'JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS', 'CONSENSO FEDERAL',
       'UNITE POR LA LIBERTAD Y LA DIGNIDAD'], dtype=object)

In [34]:
df_consenso_federal_gen = agrupar_datos(votos_generales, 'CONSENSO FEDERAL', 'Votos_consenso_federal_gen')
df_fit_gen = agrupar_datos(votos_generales, 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'Votos_fit_gen')
df_todos_gen = agrupar_datos(votos_generales, 'FRENTE DE TODOS', 'Votos_Frente_todos_gen')
df_nos_gen = agrupar_datos(votos_generales, 'FRENTE NOS', 'Votos_NOS_gen')
df_juntos_cambio_gen = agrupar_datos(votos_generales, 'JUNTOS POR EL CAMBIO', 'Votos_juntos_cambio_gen')
df_unite_gen = agrupar_datos(votos_generales, 'UNITE POR LA LIBERTAD Y LA DIGNIDAD', 'Votos_unite_gen')

In [35]:
data_frames = [df_consenso_federal_gen, df_fit_gen, df_todos_gen, df_nos_gen, df_juntos_cambio_gen, df_unite_gen]

In [36]:
df = reduce(lambda  left,right: pd.merge(left,right,on=['CODIGO_MESA'],
                                            how='right'), data_frames)
df

,CODIGO_MESA,Votos_consenso_federal_gen,Votos_fit_gen,Votos_Frente_todos_gen,Votos_NOS_gen,Votos_juntos_cambio_gen,Votos_unite_gen
0,0100100001X,14,14,114,3,115,5
1,0100100002X,9,11,105,2,111,7
2,0100100003X,14,11,122,2,105,4
3,0100100004X,13,8,117,5,106,11
4,0100100005X,19,8,127,1,108,3
...,...,...,...,...,...,...,...
97277,2400500480X,9,3,122,23,66,4
97278,2400500481X,19,4,125,7,49,2
97279,2400500482X,16,2,134,18,46,2
97280,2400500483X,11,3,136,13,58,4


In [37]:
mesas_generales

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,votos_blanco_gen
0,1,1001,01001000001,0100100001X,100000000000,5
1,1,1001,01001000001,0100100002X,100000000000,4
2,1,1001,01001000001,0100100003X,100000000000,1
3,1,1001,01001000001,0100100004X,100000000000,5
4,1,1001,01001000001,0100100005X,100000000000,2
...,...,...,...,...,...,...
97277,24,24005,24005000021,2400500480X,100000000000,5
97278,24,24005,24005000021,2400500481X,100000000000,5
97279,24,24005,24005000021,2400500482X,100000000000,5
97280,24,24005,24005000021,2400500483X,100000000000,1


In [38]:
df_generales = pd.merge(mesas_generales, df, how = 'right', on = 'CODIGO_MESA')

In [39]:
df_generales = df_generales.drop(columns = ['CODIGO_DISTRITO', 'CODIGO_SECCION', 'CODIGO_CIRCUITO', 'CODIGO_CATEGORIA'])

In [40]:
df_generales.head()

,CODIGO_MESA,votos_blanco_gen,Votos_consenso_federal_gen,Votos_fit_gen,Votos_Frente_todos_gen,Votos_NOS_gen,Votos_juntos_cambio_gen,Votos_unite_gen
0,0100100001X,5,14,14,114,3,115,5
1,0100100002X,4,9,11,105,2,111,7
2,0100100003X,1,14,11,122,2,105,4
3,0100100004X,5,13,8,117,5,106,11
4,0100100005X,2,19,8,127,1,108,3


In [41]:
df_generales['Total_votos_afirmativos_gen'] = df_generales.iloc[:,2:].sum(axis = 1)
df_generales['Total_votos_validos_gen'] =df_generales.iloc[:,0:8].sum(axis = 1)

In [42]:
df_generales

,CODIGO_MESA,votos_blanco_gen,Votos_consenso_federal_gen,Votos_fit_gen,Votos_Frente_todos_gen,Votos_NOS_gen,Votos_juntos_cambio_gen,Votos_unite_gen,Total_votos_afirmativos_gen,Total_votos_validos_gen
0,0100100001X,5,14,14,114,3,115,5,265,270
1,0100100002X,4,9,11,105,2,111,7,245,249
2,0100100003X,1,14,11,122,2,105,4,258,259
3,0100100004X,5,13,8,117,5,106,11,260,265
4,0100100005X,2,19,8,127,1,108,3,266,268
...,...,...,...,...,...,...,...,...,...,...
97277,2400500480X,5,9,3,122,23,66,4,227,232
97278,2400500481X,5,19,4,125,7,49,2,206,211
97279,2400500482X,5,16,2,134,18,46,2,218,223
97280,2400500483X,1,11,3,136,13,58,4,225,226


***Calculamos los porcentajes que sacó cada agrupación***

In [43]:
df_generales.columns

Index(['CODIGO_MESA', 'votos_blanco_gen', 'Votos_consenso_federal_gen',
       'Votos_fit_gen', 'Votos_Frente_todos_gen', 'Votos_NOS_gen',
       'Votos_juntos_cambio_gen', 'Votos_unite_gen',
       'Total_votos_afirmativos_gen', 'Total_votos_validos_gen'],
      dtype='object')

In [44]:
porcentaje = lambda x,y: x/y

df_generales.loc[:,'por_consenso_federal_gen'] = porcentaje(df_generales['Votos_consenso_federal_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_fit_gen'] = porcentaje(df_generales['Votos_fit_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_juntos_cambio_gen'] = porcentaje(df_generales['Votos_juntos_cambio_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_frente_todos_gen'] = porcentaje(df_generales['Votos_Frente_todos_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_NOS_gen'] = porcentaje(df_generales['Votos_NOS_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_Unite_gen'] = porcentaje(df_generales['Votos_unite_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_blancos_gen'] = porcentaje(df_generales['votos_blanco_gen'],df_generales['Total_votos_validos_gen'])



In [45]:
print(len(df_votos_paso)) 
print(len(df_generales))

98828
97282


In [47]:
df_limpio = df_generales.merge(df_votos_paso, on = 'CODIGO_MESA',
                       how = 'right', sort = False)
df_limpio

,CODIGO_MESA,votos_blanco_gen,Votos_consenso_federal_gen,Votos_fit_gen,Votos_Frente_todos_gen,Votos_NOS_gen,Votos_juntos_cambio_gen,Votos_unite_gen,Total_votos_afirmativos_gen,Total_votos_validos_gen,...,por_fit_paso,por_MAS_paso,por_Frente_todos_paso,por_frente_patriota_paso,por_juntos_cambio_paso,por_accion_vecinal_paso,por_autonomista_paso,por_unite_paso,por_NOS_paso,por_votos_blanco_paso
0,0100100001X,5.0,14.0,14.0,114.0,3.0,115.0,5.0,265.0,270.0,...,0.063291,0.004219,0.426160,0.008439,0.358650,0.004219,0.000000,0.016878,0.021097,0.012658
1,0100100002X,4.0,9.0,11.0,105.0,2.0,111.0,7.0,245.0,249.0,...,0.038298,0.025532,0.421277,0.004255,0.378723,0.000000,0.000000,0.038298,0.034043,0.000000
2,0100100003X,1.0,14.0,11.0,122.0,2.0,105.0,4.0,258.0,259.0,...,0.060484,0.016129,0.427419,0.000000,0.379032,0.000000,0.000000,0.024194,0.012097,0.024194
3,0100100004X,5.0,13.0,8.0,117.0,5.0,106.0,11.0,260.0,265.0,...,0.036000,0.004000,0.444000,0.000000,0.320000,0.000000,0.000000,0.064000,0.012000,0.036000
4,0100100005X,2.0,19.0,8.0,127.0,1.0,108.0,3.0,266.0,268.0,...,0.038760,0.019380,0.422481,0.000000,0.360465,0.000000,0.000000,0.038760,0.007752,0.038760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98823,2400200317X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.022222,0.008889,0.524444,0.000000,0.226667,0.000000,0.000000,0.048889,0.040000,0.031111
98824,2400200443X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.017621,0.000000,0.691630,0.000000,0.101322,0.000000,0.000000,0.026432,0.074890,0.030837
98825,2400200455X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.014286,0.014286,0.635714,0.000000,0.185714,0.007143,0.007143,0.021429,0.057143,0.000000
98826,2400200457X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_limpio['Total_votos_validos_mesa'] = df_limpio['Total_votos_validos_gen'] + df_limpio['Total_votos_validos']

In [50]:
df_limpio = df_limpio[df_limpio.Total_votos_validos_mesa >  0 ]

In [51]:
df_limpio = df_limpio.dropna()

,CODIGO_MESA,votos_blanco_gen,Votos_consenso_federal_gen,Votos_fit_gen,Votos_Frente_todos_gen,Votos_NOS_gen,Votos_juntos_cambio_gen,Votos_unite_gen,Total_votos_afirmativos_gen,Total_votos_validos_gen,...,por_MAS_paso,por_Frente_todos_paso,por_frente_patriota_paso,por_juntos_cambio_paso,por_accion_vecinal_paso,por_autonomista_paso,por_unite_paso,por_NOS_paso,por_votos_blanco_paso,Total_votos_validos_mesa
0,0100100001X,5.0,14.0,14.0,114.0,3.0,115.0,5.0,265.0,270.0,...,0.004219,0.426160,0.008439,0.358650,0.004219,0.000000,0.016878,0.021097,0.012658,507.0
1,0100100002X,4.0,9.0,11.0,105.0,2.0,111.0,7.0,245.0,249.0,...,0.025532,0.421277,0.004255,0.378723,0.000000,0.000000,0.038298,0.034043,0.000000,484.0
2,0100100003X,1.0,14.0,11.0,122.0,2.0,105.0,4.0,258.0,259.0,...,0.016129,0.427419,0.000000,0.379032,0.000000,0.000000,0.024194,0.012097,0.024194,507.0
3,0100100004X,5.0,13.0,8.0,117.0,5.0,106.0,11.0,260.0,265.0,...,0.004000,0.444000,0.000000,0.320000,0.000000,0.000000,0.064000,0.012000,0.036000,515.0
4,0100100005X,2.0,19.0,8.0,127.0,1.0,108.0,3.0,266.0,268.0,...,0.019380,0.422481,0.000000,0.360465,0.000000,0.000000,0.038760,0.007752,0.038760,526.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96048,2400500480X,5.0,9.0,3.0,122.0,23.0,66.0,4.0,227.0,232.0,...,0.000000,0.470046,0.004608,0.248848,0.000000,0.004608,0.009217,0.142857,0.023041,449.0
96049,2400500481X,5.0,19.0,4.0,125.0,7.0,49.0,2.0,206.0,211.0,...,0.005236,0.617801,0.005236,0.151832,0.000000,0.000000,0.015707,0.052356,0.005236,402.0
96050,2400500482X,5.0,16.0,2.0,134.0,18.0,46.0,2.0,218.0,223.0,...,0.005155,0.597938,0.005155,0.159794,0.000000,0.005155,0.010309,0.077320,0.036082,417.0
96051,2400500483X,1.0,11.0,3.0,136.0,13.0,58.0,4.0,225.0,226.0,...,0.000000,0.614634,0.014634,0.165854,0.000000,0.000000,0.000000,0.068293,0.058537,431.0


****Ya finalizado el pre-procesamiento, podemos definir las variables dependientes y las independientes****

In [52]:
df_limpio.columns

Index(['CODIGO_MESA', 'votos_blanco_gen', 'Votos_consenso_federal_gen',
       'Votos_fit_gen', 'Votos_Frente_todos_gen', 'Votos_NOS_gen',
       'Votos_juntos_cambio_gen', 'Votos_unite_gen',
       'Total_votos_afirmativos_gen', 'Total_votos_validos_gen',
       'por_consenso_federal_gen', 'por_fit_gen', 'por_juntos_cambio_gen',
       'por_frente_todos_gen', 'por_NOS_gen', 'por_Unite_gen',
       'por_blancos_gen', 'votos_blanco_paso', 'Votos_consenso_federal_paso',
       'Votos_MAS_paso', 'Votos_fit_paso', 'Votos_Frente_todos_paso',
       'Votos_NOS_paso', 'Votos_frente_patriota_paso',
       'Votos_juntos_cambio_paso', 'Votos_accion_vecinal_paso',
       'Votos_autonomista_paso', 'Votos_unite_paso', 'Total_votos_afrimativos',
       'Total_votos_validos', 'por_consenso_federal_paso', 'por_fit_paso',
       'por_MAS_paso', 'por_Frente_todos_paso', 'por_frente_patriota_paso',
       'por_juntos_cambio_paso', 'por_accion_vecinal_paso',
       'por_autonomista_paso', 'por_unite_paso'

In [54]:
X = df_limpio[['por_consenso_federal_paso', 'por_fit_paso', 'por_MAS_paso',
       'por_Frente_todos_paso', 'por_frente_patriota_paso',
       'por_juntos_cambio_paso', 'por_accion_vecinal_paso',
       'por_autonomista_paso', 'por_unite_paso', 'por_NOS_paso',
       'por_votos_blanco_paso']]

y = df_limpio[['por_consenso_federal_gen', 'por_fit_gen',
       'por_juntos_cambio_gen', 'por_frente_todos_gen', 'por_NOS_gen',
       'por_Unite_gen', 'por_blancos_gen']]

In [ ]:
X.head()

In [ ]:
y.head()

***Importamos y entrenamos el modelo***

In [82]:
import statsmodels.api as sm
r2 = []

def get_ols_regression(y,column_name,X):
    model = sm.OLS(y[column_name], X).fit()
    r_squared = model.rsquared
    r2.append(r_squared)
    model_summary = model.summary()
    results_as_html = model_summary.tables[1].as_html()
    table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    table.reset_index(inplace = True)
    table.rename(columns = {'coef':column_name, 'index': 'Features'}, inplace = True)
    return table

In [83]:
results_JxC = get_ols_regression(y, 'por_juntos_cambio_gen', X)
results_frente_todos = get_ols_regression(y, 'por_frente_todos_gen', X)
results_consenso_federal = get_ols_regression(y, 'por_consenso_federal_gen', X)
results_unite = get_ols_regression(y, 'por_Unite_gen', X)
results_fit = get_ols_regression(y, 'por_fit_gen', X)
results_blancos = get_ols_regression(y, 'por_blancos_gen', X)
results_nos = get_ols_regression(y, 'por_NOS_gen', X)

In [84]:
r2

[0.9012788917268684,
 0.8930067163849937,
 0.43434928881576396,
 0.18577655736822973,
 0.2640928441108068,
 0.05254771995362306,
 0.4278296871770809]

In [59]:
def features_repetidas():
    features = ['Consenso Federal', 'FIT', 'MAS',
       'Frente de Todos', 'Frente Patriota', 'Juntos por el Cambio',
       'Acción Vecinal', 'Partido Autonomista', 'Unite',
       'Frente Nos', 'Votos en Blanco']
    
    total_votos_paso = [df_limpio['Votos_consenso_federal_paso'].sum(), df_limpio['Votos_fit_paso'].sum(),
                        df_limpio['Votos_MAS_paso'].sum(), df_limpio['Votos_Frente_todos_paso'].sum(),
                        df_limpio['Votos_frente_patriota_paso'].sum(), df_limpio['Votos_juntos_cambio_paso'].sum(), df_limpio['Votos_accion_vecinal_paso'].sum(),
                       df_limpio['Votos_autonomista_paso'].sum(), df_limpio['Votos_unite_paso'].sum(), df_limpio['Votos_NOS_paso'].sum(), df_limpio['votos_blanco_paso'].sum()]
    
    features = pd.DataFrame(features, columns = ['Features'])
    total_votos_paso = pd.DataFrame(total_votos_paso , columns = ['Total_votos_paso'])
    
    features_votos_paso = features.join(total_votos_paso)
    features_votos_paso = pd.concat([features_votos_paso]*7 , ignore_index = True)
    
    features_votos_paso.reset_index(drop = True , inplace = True)
    
    features_votos_paso['Coefficients'] = pd.Series(dtype=int)
    
    features_votos_paso.reset_index(drop = True , inplace=True)
    return features_votos_paso
features_votos_paso = features_repetidas()

In [60]:
features_votos_paso

,Features,Total_votos_paso,Coefficients
0,Consenso Federal,1954251.0,NaN
1,FIT,677551.0,NaN
2,MAS,168633.0,NaN
3,Frente de Todos,11256431.0,NaN
4,Frente Patriota,56923.0,NaN
...,...,...,...
72,Acción Vecinal,35515.0,NaN
73,Partido Autonomista,31240.0,NaN
74,Unite,520364.0,NaN
75,Frente Nos,625458.0,NaN


In [65]:
def y_repetidas():
    
    ys_rep = ['Juntos por el Cambio',
       'Frente de Todos', 'Consenso Federal', 'Unite',
       'FIT', 'Frente Nos','Votos en blanco']
    
    total_votos_gen = [df_limpio['Votos_juntos_cambio_gen'].sum(), df_limpio['Votos_Frente_todos_gen'].sum(),
                        df_limpio['Votos_consenso_federal_gen'].sum(), df_limpio['Votos_unite_gen'].sum(),
                        df_limpio['Votos_fit_gen'].sum(), df_limpio['Votos_NOS_gen'].sum(), df_limpio['votos_blanco_gen'].sum()]
    

    ys_rep = pd.Series(ys_rep)
    ys_rep = ys_rep.repeat(11)
    
    total_votos_gen = pd.Series(total_votos_gen)
    total_votos_gen = total_votos_gen.repeat(11)
    
    ys_rep = pd.DataFrame(ys_rep , columns = ['ys_rep'])
    ys_rep.reset_index(drop = True, inplace = True)
    
    total_votos_gen = pd.DataFrame(total_votos_gen , columns = ['Total_votos_gen'])
    total_votos_gen.reset_index(drop = True, inplace = True)
    
    
    ys_total = ys_rep.join(total_votos_gen)
    
    ys_total = ys_total.reset_index(drop = True)
    
    return ys_total

ys_total = y_repetidas()
ys_total

,ys_rep,Total_votos_gen
0,Juntos por el Cambio,10152714.0
1,Juntos por el Cambio,10152714.0
2,Juntos por el Cambio,10152714.0
3,Juntos por el Cambio,10152714.0
4,Juntos por el Cambio,10152714.0
...,...,...
72,Votos en blanco,384458.0
73,Votos en blanco,384458.0
74,Votos en blanco,384458.0
75,Votos en blanco,384458.0


In [66]:
results_table = features_votos_paso.join(ys_total)
results_table.ys_rep.unique()

array(['Juntos por el Cambio', 'Frente de Todos', 'Consenso Federal',
       'Unite', 'FIT', 'Frente Nos', 'Votos en blanco'], dtype=object)

In [68]:
def insert_coefficients(slice1, slice2, df, column_name):
    
    results_table.loc[slice1 : slice2, 'Coefficients'] = df[column_name].values.reshape(-1,1)

    return results_table

results_table = insert_coefficients(0,10, results_JxC, 'por_juntos_cambio_gen')
results_table = insert_coefficients(11,21, results_frente_todos, 'por_frente_todos_gen')
results_table = insert_coefficients(22,32, results_consenso_federal, 'por_consenso_federal_gen')
results_table = insert_coefficients(33,43, results_unite, 'por_Unite_gen')
results_table = insert_coefficients(44,54, results_fit, 'por_fit_gen')
results_table = insert_coefficients(55,65, results_nos, 'por_NOS_gen')
results_table = insert_coefficients(66,76, results_blancos, 'por_blancos_gen')

,Features,Total_votos_paso,Coefficients,ys_rep,Total_votos_gen
0,Consenso Federal,1954251.0,0.3879,Juntos por el Cambio,10152714.0
1,FIT,677551.0,0.0232,Juntos por el Cambio,10152714.0
2,MAS,168633.0,-0.0640,Juntos por el Cambio,10152714.0
3,Frente de Todos,11256431.0,0.0022,Juntos por el Cambio,10152714.0
4,Frente Patriota,56923.0,0.1503,Juntos por el Cambio,10152714.0


In [70]:
results_table.tail()

,Features,Total_votos_paso,Coefficients,ys_rep,Total_votos_gen
72,Acción Vecinal,35515.0,0.1987,Votos en blanco,384458.0
73,Partido Autonomista,31240.0,0.0588,Votos en blanco,384458.0
74,Unite,520364.0,-0.0233,Votos en blanco,384458.0
75,Frente Nos,625458.0,-0.0250,Votos en blanco,384458.0
76,Votos en Blanco,736770.0,0.1545,Votos en blanco,384458.0


In [69]:
results_table[results_table.Features == 'Consenso Federal']

,Features,Total_votos_paso,Coefficients,ys_rep,Total_votos_gen
0,Consenso Federal,1954251.0,0.3879,Juntos por el Cambio,10152714.0
11,Consenso Federal,1954251.0,0.1273,Frente de Todos,11974998.0
22,Consenso Federal,1954251.0,0.4291,Consenso Federal,1546884.0
33,Consenso Federal,1954251.0,0.0383,Unite,370372.0
44,Consenso Federal,1954251.0,0.0225,FIT,541143.0
55,Consenso Federal,1954251.0,0.0036,Frente Nos,426737.0
66,Consenso Federal,1954251.0,-0.0088,Votos en blanco,384458.0


In [ ]:
# results_table.to_csv(r'C:\Users\Administrador\Desktop\Carrera de Data Science\DS-Cor\Transferencia votos\resultados_modelo_mesas.csv')

In [ ]:
input_data = {'a': {'aa': 0.3, 'cc': 0.7,},
              'b': {'aa': 2, 'bb': 0.5,},
              'c': {'aa': 0.5, 'bb': 0.5, 'cc': 1.5,}}